# About This Assignment

Design and implement a complete **Natural Language Processing (NLP)** pipeline for
advanced sequence-to-sequence tasks using the Sherlock Holmes dataset, including:
-  text summarisation
- semantic search
- thematic analysis 

The focus is on understanding the process, implementing modular steps, and critically evaluating outcomes.

**Objective** 

To write a comprehensive report detailing the development, findings, and
results of your (NLP) pipeline, focusing on:
- How design choices influenced performance.
- Challenges encountered at each stage.
- Insights gained from the dataset and NLP methods used.
- Suggest improvements for each component of the pipeline.


# About this Data

- This collection features all the stories and novels of Sherlock Holmes by Arthur Conan Doyle. 
- Within the Sherlock folder, you'll find multiple .txt files, each containing a unique story.

# Importing neccesary libraries

In [3]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Importing the dataset

The dataset is presented as a folder containing each story individually in a txt file.

In [1]:
path = 'sherlock'

In [5]:
# list of files in the folder
files = os.listdir(path)

# Iterate over each file in the folder
for i in files:
        # Create a variable name from the file name without the extension
        var_name = i[:-4]
        
        # If the first character of the variable name is a number, add an underscore
        if var_name[0].isdigit():
            var_name = '_' + var_name

        # Read the contents of the file
        with open(os.path.join(path, i), 'r') as data:
            contents = data.read()
        
        # Assign the contents to a variable with the name of the file
        globals()[var_name] = contents


print(_3gab)
print(_3gar)






                        THE ADVENTURE OF THE THREE GABLES

                               Arthur Conan Doyle



     I don't think that any of my adventures with Mr. Sherlock Holmes
     opened quite so abruptly, or so dramatically, as that which I
     associate with The Three Gables. I had not seen Holmes for some days
     and had no idea of the new channel into which his activities had been
     directed. He was in a chatty mood that morning, however, and had just
     settled me into the well-worn low armchair on one side of the fire,
     while he had curled down with his pipe in his mouth upon the opposite
     chair, when our visitor arrived. If I had said that a mad bull had
     arrived it would give a clearer impression of what occurred.

     The door had flown open and a huge negro had burst into the room. He
     would have been a comic figure if he had not been terrific, for he
     was dressed in a very loud gray check suit with a flowing
     salmon-coloured tie. H